In [1]:
import os
import torch
import random
import numpy as np
import json
import torch.nn as nn
from predict import predict_from_dataset
from utils import seed_everything, label_enc, print_evaluation_metrics, load_data_owner_dataset, get_device
import pandas as pd
from collections import Counter

In [2]:
def loadmodel(name):
    x = open('model_parameters/'+ name+'.json')
    data = json.load(x)
    model_structure = data['model_structure']
    model_parameters_names = data['model_parameters_names']
    model_parameters_vals_= data['model_parameters_vals']
    model_parameters_vals =[]
    for i in model_parameters_vals_:
        tmp=torch.tensor(i)
        tmp2=torch.rand(tmp.shape)*0.0000001
        tmp3=torch.add(tmp,tmp2)
        model_parameters_vals.append(tmp3.tolist())
    model_parameters_names = [f'[{name.split(".")[0]}].{name.split(".")[1]}'
                                   for name in model_parameters_names.split('|')]
    received_model = nn.Sequential(
    *[eval('nn.' + layer) for layer in model_structure.split('|')]
    )
    for parameters_name, parameters_val in zip(model_parameters_names, model_parameters_vals):
        exec(f'received_model{parameters_name} = nn.Parameter(torch.tensor(parameters_val))')
    for param in received_model.parameters():
        param.requires_grad = False
    return received_model.to('cuda')

In [92]:

def checksame():
    hh=['model_A','model_B','model_C','model_D','model_E','model_F','model_G','model_T1','model_T2','model_T3']
    for i in hh:
        y=torch.load('../../saved_models/'+ i+'.pt').to('cuda')
        x=loadmodel(i)
        random_input = torch.randn(32, 768).to('cuda')
        x.eval()
        x_ = x(random_input)
        y.eval()
        y_= y(random_input)
        print(i)
        if (x_ == y_).all():
            print('Success! Two models have the same output')
        else:
            print('Fail! Please check if any of the parameters are different')
checksame()

model_A
Success! Two models have the same output
model_B
Success! Two models have the same output
model_C
Success! Two models have the same output
model_D
Success! Two models have the same output
model_E
Success! Two models have the same output
model_F
Success! Two models have the same output
model_G
Success! Two models have the same output
model_T1
Success! Two models have the same output
model_T2
Success! Two models have the same output
model_T3
Success! Two models have the same output


In [7]:
def loadmodels(names):
    all=[]
    for i in names:
        all.append(loadmodel(i))
    return all


In [8]:
dataset_path = '../../Datasets/CIFAR10'

In [9]:
def ensemble(models,path,device):
    for i in models:
        return predict_from_dataset(path,'A',"Z",['plane','car', 'ship', 'horse', 'truck', 'other'],i,device)
    

In [10]:
def final(path):
    models= loadmodels(["model_A"])
    y= ensemble(models,dataset_path,'cuda')
    return y

In [7]:
print(final(dataset_path))

<torch.utils.data.dataloader.DataLoader object at 0x000001BE30CCADF0> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=6, bias=True)
)


100%|██████████| 79/79 [01:07<00:00,  1.17it/s]


<torch.utils.data.dataloader.DataLoader object at 0x000001BE3089BF40> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=6, bias=True)
)


100%|██████████| 66/66 [00:55<00:00,  1.19it/s]

['plane', 'plane', 'plane', 'plane', 'ship', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'car', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'plane', 'pl

In [8]:
x = open('../../dataOwnerLabelInfo.json')

In [9]:
data= json.load(x)

In [13]:
model_labels=[]
for i in data:
    if(data[i][0]=='trainer'):
        model_labels.append(data[i][1].split(','))

{'A': ['trainer', 'plane, car, ship, horse, truck, other'], 'B': ['trainer', 'truck, car, other'], 'C': ['trainer', 'plane, truck, ship, other'], 'D': ['trainer', 'ship, plane'], 'E': ['trainer', 'plane, car, ship, other'], 'F': ['trainer', 'cat, dog, other'], 'G': ['trainer', 'truck, other'], 'T1': ['trainer', 'truck, car'], 'T2': ['trainer', 'plane, ship'], 'T3': ['trainer', 'cat, dog'], 'X': ['client', 'truck'], 'Y': ['client', 'truck'], 'Z': ['client', 'plane, ship'], 'T4': ['client', 'horse']}


In [48]:
img_path = dataset_path + '/images'

In [55]:
from utils import seed_everything, label_enc, print_evaluation_metrics, load_data_owner_dataset, get_device
client_dataset = load_data_owner_dataset(dataset_path, 'Z')

In [77]:
print(client_dataset.groupby('label_name').get_group('ship'))


                                        image  label label_name
2500  ../../Datasets/CIFAR10/images/08359.png      8       ship
2501  ../../Datasets/CIFAR10/images/05788.png      8       ship
2502  ../../Datasets/CIFAR10/images/28007.png      8       ship
2503  ../../Datasets/CIFAR10/images/20396.png      8       ship
2504  ../../Datasets/CIFAR10/images/41241.png      8       ship
...                                       ...    ...        ...
4595  ../../Datasets/CIFAR10/images/49969.png      8       ship
4596  ../../Datasets/CIFAR10/images/53038.png      8       ship
4597  ../../Datasets/CIFAR10/images/45698.png      8       ship
4598  ../../Datasets/CIFAR10/images/29079.png      8       ship
4599  ../../Datasets/CIFAR10/images/40779.png      8       ship

[2100 rows x 3 columns]


In [59]:
client_dataset.label_name.unique()

array(['plane', 'ship'], dtype=object)

In [3]:
def ensemble_model(client):        
    dataset_path = '../../Datasets/CIFAR10'
    #corresponding labels for each model
    x = open('../../dataOwnerLabelInfo.json')
    data= json.load(x)
    model_names=[]
    model_labels=[]
    for i in data:
        if(data[i][0]=='trainer'):
            model_names.append(('model_'+i)) 
            model_labels.append(data[i][1].split(', '))
    all_models=[]
    for i in model_names:
        all_models.append(loadmodel(i))
    df = pd.DataFrame()
    for i in range(len(all_models)):
        name= model_names[i].split('_')[1]
        tmp=pd.Series(predict_from_dataset(dataset_path,name,client,model_labels[i],all_models[i],'cuda'))
        df[model_names[i]]=tmp
    tmp = df.T
    result=[]
    for x in tmp.columns:
        result.append(tmp[x].value_counts().idxmax())
    client_dataset = load_data_owner_dataset(dataset_path, client)
    client_dataset['prediction'] = result
    client_dataset.to_csv(f'Client_{client}.csv')


In [4]:
tmp=ensemble_model('Z')

<torch.utils.data.dataloader.DataLoader object at 0x000002913F14FE80> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=6, bias=True)
)


100%|██████████| 79/79 [01:06<00:00,  1.19it/s]


2500
<torch.utils.data.dataloader.DataLoader object at 0x000002913F28AFA0> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=6, bias=True)
)


100%|██████████| 66/66 [00:57<00:00,  1.15it/s]


4600


...

100%|██████████| 79/79 [01:10<00:00,  1.13it/s]


2500
<torch.utils.data.dataloader.DataLoader object at 0x000002913F16C4C0> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=64, bias=True)
  (2): GELU()
  (3): Linear(in_features=64, out_features=2, bias=True)
)


100%|██████████| 66/66 [00:58<00:00,  1.13it/s]


4600


C:\Users\haichen\AppData\Local\Temp\ipykernel_43412\2569488414.py:18: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tmp=pd.Series(predict_from_dataset(dataset_path,name,client,model_labels[i],all_models[i],'cuda'))


In [79]:
tmp= tmp.T